In [1]:
import os
from pathlib import Path

import pandas as pd

from util import with_spotify_track_metadata

diff_dataset = True
data_path = Path("../../data").resolve()
os.makedirs(data_path, exist_ok=True)

/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def fetch_song_chart_by_month():
    """Chart 2000 - 2021: the top 50 chart for every month from Jan 2000 to Jun 2021
    Note: song and albums ranked weekly by indicative revenue globally over a certain period. Indicative Revenue is a model for global sales volume, adjusted for inflation and currency exchange, and using IFPI estimated annual revenue for the recorded music industry in each country.
    #songs_chart_by_month = pd.read_csv("https://chart2000.com/data/chart2000-songmonth-0-3-0063.csv")
    """
    return pd.read_csv("/Users/pez/Downloads/chart2000-songmonth-0-3-0063.csv")


songs_chart_by_month = fetch_song_chart_by_month()
songs_chart_by_month = with_spotify_track_metadata(songs_chart_by_month)
#songs_chart_by_month.to_parquet(data_path / "raw/song_chart_by_month.pq", index=False)
#songs_chart_by_month.to_csv(data_path / "song_chart_by_month.csv.gz", compression="gzip", index=False)

In [3]:
songs_chart_by_month = fetch_song_chart_by_month()
bsb_songs_chart = songs_chart_by_month[songs_chart_by_month["artist"].str.contains("Backstreet Boys")]
bsb_songs_chart = with_spotify_track_metadata(bsb_songs_chart)
bsb_songs_chart

/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/pez/.conda/envs/dataspell-2/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,au,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
43,Jan 2000,44,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1457.522,16,97,-,-,-,-,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
56,Feb 2000,7,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2914.772,10,3,3,-,1,20,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
102,Mar 2000,3,The Backstreet Boys,Show Me The Meaning Of Being Lonely,3678.622,6,3,4,-,1,19,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
158,Apr 2000,9,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2945.213,6,28,22,-,-,23,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
219,May 2000,20,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1919.076,19,64,66,-,-,41,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
275,Jun 2000,26,The Backstreet Boys,The One,1857.997,31,8,15,-,-,-,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
321,Jul 2000,22,The Backstreet Boys,The One,1997.071,30,15,41,-,-,41,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
504,Nov 2000,5,The Backstreet Boys,Shape Of My Heart,3351.040,10,4,2,-,1,5,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
552,Dec 2000,3,The Backstreet Boys,Shape Of My Heart,3408.347,9,20,3,-,1,11,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
619,Jan 2001,20,The Backstreet Boys,Shape Of My Heart,2208.448,19,49,14,-,-,17,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day


In [5]:
import json
from time import sleep

from util import spotify


def fetch_bulk_track_features(track_ids):
    fetched_tracks = []
    for i in range(0, len(track_ids), 50):
        chunk = track_ids[i:i + 50]
        if i % 500 == 0:
            print(f"fetching track metadata: {i} ({i / len(track_ids) * 100:.2f})")
        tracks = spotify.tracks(chunk)["tracks"]
        sleep(.50)
        track_audio_features = spotify.audio_features(chunk)
        sleep(.50)
        for track, audio_features in zip(tracks, track_audio_features):
            track_ = {
                "id": track["id"],
                "name": track["name"],
                "album": track["album"]["name"],
                "album_id": track["album"]["id"],
                "artists": json.dumps([a["name"] for a in track["artists"]]),
                "artist_ids": json.dumps([a["id"] for a in track["artists"]]),
                "track_number": track["track_number"],
                "disc_number": track["disc_number"],
                "explicit": track["explicit"],
                "duration_ms": track["duration_ms"],
                "year": int(track["album"]["release_date"][0:4]),
                "release_date": track["album"]["release_date"],
                "popularity": track["popularity"],
                "audio_features": 0,
                "isrc": track.get("external_ids", {}).get("isrc")
            }
            if audio_features is not None:
                track_.update({
                    "audio_features": 1,
                    "danceability": audio_features["danceability"],
                    "energy": audio_features["energy"],
                    "key": audio_features["key"],
                    "loudness": audio_features["loudness"],
                    "mode": audio_features["mode"],
                    "speechiness": audio_features["speechiness"],
                    "acousticness": audio_features["acousticness"],
                    "instrumentalness": audio_features["instrumentalness"],
                    "liveness": audio_features["liveness"],
                    "valence": audio_features["valence"],
                    "tempo": audio_features["tempo"],
                    "time_signature": audio_features["time_signature"],
                })
            fetched_tracks.append(track_)
    return pd.DataFrame(fetched_tracks)

track_ids = bsb_songs_chart["spotify_track_id"].tolist()
tracks_df = fetch_bulk_track_features(track_ids)
tracks_df

fetching track metadata: 0 (0.00)


,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
1,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
2,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
3,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
4,3BsaRV5QIulYz2lV9WWa8T,Show Me the Meaning of Being Lonely,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",3,1,False,234960,...,6,-5.088,0,0.0252,0.2310,0,0.0765,0.683,167.998,4
5,44kgtDJyAFv4Z01z7bIxco,The One,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",8,1,False,226933,...,5,-4.227,0,0.0348,0.0646,0,0.1640,0.621,110.031,4
6,44kgtDJyAFv4Z01z7bIxco,The One,Millennium,5ySxm9hxBNss01WCL7GLyQ,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",8,1,False,226933,...,5,-4.227,0,0.0348,0.0646,0,0.1640,0.621,110.031,4
7,35o9a4iAfLl5jRmqMX9c1D,Shape of My Heart,Black & Blue,41zXjyVr6dzmchWf8tv3UO,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",2,1,False,230093,...,9,-4.353,1,0.0296,0.2520,0,0.1590,0.518,96.102,4
8,35o9a4iAfLl5jRmqMX9c1D,Shape of My Heart,Black & Blue,41zXjyVr6dzmchWf8tv3UO,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",2,1,False,230093,...,9,-4.353,1,0.0296,0.2520,0,0.1590,0.518,96.102,4
9,35o9a4iAfLl5jRmqMX9c1D,Shape of My Heart,Black & Blue,41zXjyVr6dzmchWf8tv3UO,"[""Backstreet Boys""]","[""5rSXSAkZ67PYJSvpUpkOr7""]",2,1,False,230093,...,9,-4.353,1,0.0296,0.2520,0,0.1590,0.518,96.102,4


In [6]:
bsb_songs_chart

,month,position,artist,song,indicativerevenue,us,uk,de,fr,ca,au,spotify_track_id,spotify_artist_id,spotify_album_id,spotify_track_popularity,spotify_track_explicit,spotify_track_duration_ms,spotify_track_album_release_date,spotify_track_album_release_date_precision
43,Jan 2000,44,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1457.522,16,97,-,-,-,-,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
56,Feb 2000,7,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2914.772,10,3,3,-,1,20,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
102,Mar 2000,3,The Backstreet Boys,Show Me The Meaning Of Being Lonely,3678.622,6,3,4,-,1,19,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
158,Apr 2000,9,The Backstreet Boys,Show Me The Meaning Of Being Lonely,2945.213,6,28,22,-,-,23,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
219,May 2000,20,The Backstreet Boys,Show Me The Meaning Of Being Lonely,1919.076,19,64,66,-,-,41,3BsaRV5QIulYz2lV9WWa8T,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,71.0,False,234960.0,1999-05-18,day
275,Jun 2000,26,The Backstreet Boys,The One,1857.997,31,8,15,-,-,-,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
321,Jul 2000,22,The Backstreet Boys,The One,1997.071,30,15,41,-,-,41,44kgtDJyAFv4Z01z7bIxco,5rSXSAkZ67PYJSvpUpkOr7,5ySxm9hxBNss01WCL7GLyQ,62.0,False,226933.0,1999-05-18,day
504,Nov 2000,5,The Backstreet Boys,Shape Of My Heart,3351.040,10,4,2,-,1,5,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
552,Dec 2000,3,The Backstreet Boys,Shape Of My Heart,3408.347,9,20,3,-,1,11,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
619,Jan 2001,20,The Backstreet Boys,Shape Of My Heart,2208.448,19,49,14,-,-,17,35o9a4iAfLl5jRmqMX9c1D,5rSXSAkZ67PYJSvpUpkOr7,41zXjyVr6dzmchWf8tv3UO,74.0,False,230093.0,2000-11-21,day
